In [88]:
!pip install fasttext
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 3.8MB/s 


In [50]:
import pandas as pd
import numpy as np
import fasttext.util
from collections import defaultdict
import ast

In [52]:
fasttext.util.download_model('hi', if_exists='ignore')  # Hindi
ft = fasttext.load_model('cc.hi.300.bin')

In [99]:
train_file = 'train.csv'
val_file = 'val.csv'

In [100]:
train_data = pd.read_csv(train_file, header=0, index_col=0)
val_data = pd.read_csv(val_file, header=0, index_col=0)

In [101]:
print(train_data.shape)
train_data.head()

(5727, 10)


,Post,Labels Set,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post
Unique ID,,,,,,,,,,
1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive",[],[],[],[],"['🙏', '🙏']",[],[],मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...
2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile,[],['https://t.co/8iy2MJSBAs'],[],[],[],[],[],सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...
3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile,[],"['https://t.co/G945HvzM0Z', 'https://t.co/KfH7...",[],['LIVE'],[],[],[],"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील..."
4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं...,"defamation,offensive",[],[],['@prabhav218'],[],[],[],[],साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते...
5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स...,non-hostile,[],['https://t.co/4e6lysg0VR'],[],['unlock4guidelines'],[],[],[],- अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंब...


In [102]:
print(val_data.shape)
val_data.head()

(811, 10)


,Post,Labels Set,emails,urls,mentions,hashtags,emojis,emoticons,reserved_words,Filtered_Post
Unique ID,,,,,,,,,,
1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...,non-hostile,[],[],[],[],[],[],[],दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए दे...
2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...,defamation,[],[],[],[],[],[],[],भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं ...
3,कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...,non-hostile,[],['https://t.co/9rlQowAsFh'],"['@ArvindKejriwal', '@rajnathsingh', '@AmitSha...",['Delhi'],[],[],[],कोरोना से निपटने की तैयारी / दिल्ली में 10 हजा...
4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...,non-hostile,[],['https://t.co/ZvKgxk6dbd'],[],[],[],[],[],गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीत...
5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव...",non-hostile,[],['https://t.co/hxM1uNNmX2'],[],['UP'],[],[],[],"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव..."


In [256]:
mentions_dict = defaultdict(int)

for index, row in train_data.iterrows():
  mentions = ast.literal_eval(row['mentions']) 
  for m in mentions:
    mentions_dict[m]+=1


In [257]:
filtered_mentions_list = []

for key, value in mentions_dict.items():
  if value > 30:
    filtered_mentions_list.append(key)

In [258]:
mention_len = len(filtered_mentions_list)

In [259]:
hashtag_dict = defaultdict(int)

for index, row in train_data.iterrows():
  mentions = ast.literal_eval(row['hashtags']) 
  for m in mentions:
    hashtag_dict[m]+=1


In [260]:
filtered_hashtag_list = []

for key, value in hashtag_dict.items():
  if value > 5:
    filtered_hashtag_list.append(key)

In [261]:
hash_len = len(filtered_hashtag_list)

In [262]:
def generate_matrix(data):
  X = np.empty((0, 300 + 1 + hash_len + mention_len))

  for index, row in data.iterrows():
    x = np.zeros((1, 301))
    x[0, 0] = 1  # Bias Unit

    # Word Embeddings
    for word in row['Filtered_Post'].split(' '):
      x[0, 1:301]+= ft[word]

    hash_x = np.zeros((1, hash_len))
    # Hashtags
    for h in ast.literal_eval(row['hashtags']):
      if h in filtered_hashtag_list:
        hash_x[0, filtered_hashtag_list.index(h)] = 1

    
    mention_x = np.zeros((1, mention_len))
    # Mentions
    for h in ast.literal_eval(row['mentions']):
      if m in filtered_mentions_list:
        mention_x[0, filtered_mentions_list.index(m)] = 1

    x = np.hstack((x, hash_x, mention_x))
    X = np.vstack((X, x))

  return X


In [263]:
train_X = generate_matrix(train_data)
val_X = generate_matrix(val_data)

In [264]:
val_X.shape

(811, 418)

In [265]:
# Label Mapping
labels_mapping = {'defamation':0,
 'fake':1,
 'hate':2,
 'non-hostile':3,
 'offensive':4}


In [266]:
train_y = np.empty((0, 5))
for index, row in train_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  train_y = np.vstack((train_y, y))


val_y = np.empty((0, 5))
for index, row in val_data.iterrows():
  y = np.zeros((1, 5))
  for label in row['Labels Set'].split(','):
    y[0, labels_mapping[label]] = 1

  val_y = np.vstack((val_y, y))




In [267]:
y_train_defamation = train_y[:,0].astype(int)
y_val_defamation = val_y[:,0].astype(int)

In [268]:
def evaluation(y_true, y_pred):
  print("Fine Grained Accuracy = {}".format(accuracy_score(y_true, y_pred)))
  print("\n\nFine Grained Metrics\n")
  print(classification_report(y_true, y_pred))
  

**Model**

In [269]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [270]:
classifier =LogisticRegression(class_weight='balanced', solver = 'liblinear', warm_start=True)
classifier.fit(train_X, y_train_defamation)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=True)

In [271]:
predictions = classifier.predict(val_X)

In [272]:
evaluation(y_val_defamation.reshape(-1,1), predictions)

Fine Grained Accuracy = 0.7866831072749692


Fine Grained Metrics

              precision    recall  f1-score   support

           0       0.96      0.80      0.87       734
           1       0.26      0.70      0.38        77

    accuracy                           0.79       811
   macro avg       0.61      0.75      0.63       811
weighted avg       0.90      0.79      0.82       811

